The backtracking algorithm rapidly becomes prohibitively expensive as the order of the graph increases. An approximation algorithm for the Hamiltonian cycle problem would seek to make a very good attempt at finding a cycle in a short space of time. If it fails, there may have been a cycle it missed, but it is hoped that the probability of this will be small.

We shall construct a sequence of paths $P_1, P_2, \dots,$ where $P_1$ is just a single vertex $v_0$. Given a path $P_j$ from $v_0$ to $v_k$, we proceed as follows:
1.  If $P_j$ has length $n - 1$ and $v_0v_k \in E(G)$, then output a Hamiltonian cycle;
2.  If $P_j$ has length less than $n - 1$ and $v_k$ is joined to a vertex not in $P_j$, then extend the path to a path $P_{j+1}$ by picking a neighbour at random.
3.  Otherwise, construct a new path of the same length as $P_j$ in this way: select a neighbour $v_i$ of $v_k$ in $P_j$ at random. Then $P_{j+1}$ is the path $v_0 \dots v_{i-1}v_iv_kv_{k-1} \dots v_{i+1}$.

In [79]:
import random

def generate_graph(n, p):
    '''
    Generates a random graph from G(n, p).
    '''
    graph = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                graph[i][j] = 1
                graph[j][i] = 1
    return graph

def find_hamiltonian_approx(graph, T):
    '''
    Searches for a Hamiltonian cycle using a randomized approximation algorithm.
    Args:
        The adjacency matrix of the graph.
        The stopping time (maximum number of iterations).
    Returns:
        The Hamiltonian cycle if found, otherwise None.
    '''
    n = len(graph)
    if n < 3:
        return None

    # Start with a path P1 containing just a single vertex v0
    path = [0]

    for _ in range(T):
        # Check if a Hamiltonian cycle is formed
        # The path has length n-1 if it contains n vertices
        if len(path) == n:
            # Check if the last vertex is connected to the start
            if graph[path[-1]][path[0]] == 1:
                return path + [path[0]] # Return the completed cycle
            # If not, we are stuck with a Hamiltonian path. We must rotate.

        vk = path[-1] # The last vertex in the current path
        path_vertices = set(path)

        # Try to extend the path
        # Find all neighbors of vk that are NOT in the current path
        neighbors_not_in_path = [
            v for v in range(n)
            if graph[vk][v] == 1 and v not in path_vertices
        ]

        if len(path) < n and neighbors_not_in_path:
            # If there are valid neighbors, pick one at random and extend the path
            next_v = random.choice(neighbors_not_in_path)
            path.append(next_v)
            continue # Move to the next iteration

        # Otherwise, construct a new path (rotate)
        # This part is executed if the path is "stuck"
        # Find neighbors of vk that are already IN the path
        # Exclude the immediate predecessor
        predecessor = path[-2] if len(path) > 1 else -1
        neighbors_in_path = [
            v for v in range(n)
            if graph[vk][v] == 1 and v in path_vertices and v != path[0] and v != predecessor
        ]

        if neighbors_in_path:
            # Pick one of these neighbors at random
            vi = random.choice(neighbors_in_path)

            # Find the index of vi in the path
            i = path.index(vi)

            # Reverse the part of the path from vi's successor to the end (vk)
            # This creates a new path with the same vertices but a different order and endpoint.
            segment_to_reverse = path[i+1:]
            segment_to_reverse.reverse()
            path = path[:i+1] + segment_to_reverse
        else:
            # If we are stuck and cannot rotate, the algorithm has failed on this run.
            # We shall consider it a dead end for this path.
            # For simplicity, we just let the loop continue.
            pass

    return None # Algorithm failed to find a cycle within T steps

The choice of the stopping time $T$ is crucial. It represents the trade-off between the algorithm's running time and its reliability.

*   If T is too small, then the algorithm will be very fast, but it may fail to find a cycle even when one exists because it didn't have enough iterations to extend or rotate its path into the correct configuration.
*   If T is too large, then the algorithm will be more reliable, but it will be slow, defeating the purpose of using an approximation in the first place.

A good choice for $T$ is typically a polynomial function of the number of vertices $n$. For most practical purposes, a function like $T = 10 * n^2$ or $T = n^3$ is a good starting point. It ensures that the algorithm's running time remains polynomial so is much faster than the $O(n!)$ complexity of the exact backtracking algorithm, especially for larger $n$. The false failure rate should be quite low for both very high and very low probabilities.

In [80]:
# Parameters
n = 15
T = n**3

# Test on a graph that is known to be Hamiltonian
print(f"Generating a G({n}, {0.95}) random graph...")
hamiltonian_graph = generate_graph(n, 0.95)
result = find_hamiltonian_approx(hamiltonian_graph, T)

if result:
    print(f"Success! Found a Hamiltonian cycle.")
    print("Cycle:", result)
else:
    print(f"Failure. No cycle found within T iterations.")
    print("The graph may not be Hamiltonian, or T was too small.")

# Compare with a non-Hamiltonian graph (very low p)
print(f"Generating a G({n}, {0.05}) random graph...")
non_hamiltonian_graph = generate_graph(n, 0.05)
result = find_hamiltonian_approx(non_hamiltonian_graph, T)

if result:
    print(f"Success! Found a Hamiltonian cycle")
    print("Cycle:", result)
else:
    print(f"Failure, as expected. No cycle found.")

Generating a G(15, 0.95) random graph...
Success! Found a Hamiltonian cycle.
Cycle: [0, 12, 11, 3, 9, 8, 4, 10, 2, 13, 5, 14, 6, 7, 1, 0]
Generating a G(15, 0.05) random graph...
Failure, as expected. No cycle found.


When the edge probability $p$ is a fixed constant and the number of vertices $n$ is large, then the resulting graph $\mathcal{G}(n, p)$ has an expected number of edges given by $pn(n-1)/2$. In this graph, the Hamiltonian cycles are abundant. A well-known result in random graph theory is that for any fixed $p > 0$, a graph $\mathcal{G}(n, p)$ will almost surely have a Hamiltonian cycle as $n$ tends to infinity.

The approximation algorithm is highly effective. Because there are so many edges, the algorithm's path will rarely get stuck. At almost every step, the last vertex of the path will have numerous neighbors that are not yet in the path. This means the path will grow linearly.

The dominant operation will be step 2. The algorithm will likely proceed $n - 1$ times in a row, extending the path one vertex at a time until it has length $n - 1$. The "rotation" step will rarely be needed, so we are likely to find a cycle in just over $n$ steps.

To and account for somerandomness, a slightly more generous stopping time is desirable. A good choice for $T$ in this case would be a low-degree polynomial in $n$, where the dependency on $p$ is minimal. Crucially, because the graph is so dense, the stopping time does not need a strong dependency on $p$.

The actual running time of the algorithm is highly dependent on $p$, as $p$ dictates the graph's structure.

*   For low $p$, the graph is sparse, likely disconnected and almost certainly contains vertices of degree $0$ or $1$. The algorithm starts a path but gets stuck almost immediately. It cannot extend the path, and it cannot rotate it because the endpoint has no other neighbors. The algorithm will run for the full $T$ iterations where the stopping time is reached and fail.

*   For $p$ near the threshold, the graph is likely connected but sparse, with a tree-like structure. It may have a minimum degree of $2$ but contains many "split-vertices" and few paths. The algorithm can build long paths, but it gets stuck frequently. It will then need to perform many rotation operations to try to reconfigure the path. The running time is the slowest of all cases.

*   For high $p$, the graph is dense and highly connected and contains many redundant paths and numerous Hamiltonian cycles. The algorithm extends the path rapidly at almost every step and rotations are rarely needed. Hence, a cycle is found very quickly leading to the algorithm terminating in approximately $n$ steps.